# Dataset creation

## Installation

In [1]:
!pip install -q textdescriptives argilla==1.19 transformers datasets sentence-transformers


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [102]:
!python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://dmrepository.datamaran.com:8443/repository/dmPYTHON/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 41.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: en-core-web-md
    Found existing installation: en-core-web-md 3.7.0
    Uninstalling en-core-web-md-3.7.0:
      Successfully uninstalled en-core-web-md-3.7.0

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


## Imports

In [3]:
import textdescriptives as td
from datasets import load_dataset
import re
import os
import spacy
import argilla as rg
import numpy as np
import concurrent.futures
import requests
import json

/Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Connect to Argilla

In [29]:
rg.init(api_url=os.environ.get("ARGILLA_API_URL_PRE"), api_key=os.environ.get("ARGILLA_API_KEY_PRE"))
rg.set_workspace("awesome-argilla-datasets")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 rg.init(api_url=os.environ.get("ARGILLA_API_URL_PRE"), api_key=os.environ.get("ARGILLA_A     │
│   2 rg.set_workspace("awesome-argilla-datasets")                                                 │
│   3                                                                                              │
│                                                                                                  │
│ /Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9 │
│ /site-packages/argilla/client/api.py:103 in init                                                 │
│                                                                                                  │
│   100 │   │   >>> headers = {"X-Client-id":"id","X-Secret":"secret"}                             │
│   101 │   │   >>> rg.init(api_url="http://localhost:9090", api_key="4AkeAPIk3Y", extra_headers   │
│   102 │   """                                                                                    │
│ ❱ 103 │   ArgillaSingleton.init(                                                                 │
│   104 │   │   api_url=api_url,                                                                   │
│   105 │   │   api_key=api_key,                                                                   │
│   106 │   │   workspace=workspace,                                                               │
│                                                                                                  │
│ /Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9 │
│ /site-packages/argilla/client/api.py:59 in init                                                  │
│                                                                                                  │
│    56 │   ) -> Argilla:                                                                          │
│    57 │   │   cls._INSTANCE = None                                                               │
│    58 │   │                                                                                      │
│ ❱  59 │   │   cls._INSTANCE = Argilla(                                                           │
│    60 │   │   │   api_url=api_url,                                                               │
│    61 │   │   │   api_key=api_key,                                                               │
│    62 │   │   │   timeout=timeout,                                                               │
│                                                                                                  │
│ /Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9 │
│ /site-packages/argilla/client/client.py:151 in __init__                                          │
│                                                                                                  │
│   148 │   │   │   headers=headers.copy(),                                                        │
│   149 │   │   )                                                                                  │
│   150 │   │                                                                                      │
│ ❱ 151 │   │   self._user = users_api.whoami(client=self.http_client)  # .parsed                  │
│   152 │   │                                                                                      │
│   153 │   │   if not workspace and self._user.username == DEFAULT_USERNAME and DEFAULT_USERNAM   │
│   154 │   │   │   warnings.warn(                                                                 │
│                                                            

## Pre-processing

End-to-end workflow to create a dataset in Argilla with text measurements as metadata.
This aids in quickly identifying and improving potential dataset issues.

### Dataset creation

At first, we need to create a dataset in Argilla. This can either be done by loading a previous created dataset or by creating a new one. In order to avoid duplication, we will check if the dataset already exists. Additionally, we will load the markdown file that contains the dataset guidelines.


In [4]:
with open("GUIDELINES.md") as f:
    guidelines = f.read()
guidelines

"# Guidelines\n\nThe ShareGPT dataset is a dataset that was collected by public users who were using the Google Chrome extension offered by [sharegpt.com](sharegpt.com) to share their ChatGPT conversations. This data should mimic real-life usage of the model and can therefore be used to fine-tune a model for an actual scenario. Additionally, Google was accused of using this dataset as a baseline to train its [BARD](https://www.theverge.com/2023/3/29/23662621/google-bard-chatgpt-sharegpt-training-denies) model.\n\nWe decided to use a random subset of the raw version of the dataset including all conversations but we did filter out non-English conversation. The raw dataset used can be found on [the Hugging Face hub](https://huggingface.co/datasets/zetavg/ShareGPT-Processed).\n\n## Classification Tasks\n\nThe dataset aims to classify three things:\n\n1. Quality\n2. Intent\n3. Toxicity\n\n### Quality\n\nFor the quality, we have decided to define a rating question on a scale from 1 to 7. Thi

In [6]:
try:
    ds_local = rg.FeedbackDataset(
        fields=[
            rg.TextField(name="prompt", title="Prompt", use_markdown=True),
            rg.TextField(name="response", title="Response", use_markdown=True),
        ],
        questions=[
            rg.RatingQuestion(
                name="prompt-quality", 
                title="Prompt Quality",
                values=list(range(1, 8)), 
                description="How would you rate the quality of the prompt?",
            ),
            rg.LabelQuestion(
                name="prompt-intent", 
                title="Prompt Intent",
                labels=["generation", "rewrite", "extract", "closed-qa", "open-qa", "classification", "summarization", "brainstorming", "chat", "code", "other"], 
                description="What is the intent of the prompt?"
            ),
            rg.MultiLabelQuestion(
                name="response-toxicity", 
                title="Response Toxicity",
                labels=["illegal", "harmfull", "unqualified advice"], 
                description="What are the toxicities in the response (if any)?",
                required=False
            )
        ],
        guidelines=guidelines
    )
    ds_remote = ds_local.push_to_argilla("sharegpt")
except Exception as e:
    ds_remote = rg.FeedbackDataset.from_argilla("sharegpt")
ds_remote

RemoteFeedbackDataset(
   id=c082ca28-49d1-4b25-a41d-6fc143dd1e63
   name=sharegpt
   workspace=Workspace(id=462547f7-0d83-416e-8771-26bc69b63c8b, name=awesome-argilla-datasets, inserted_at=2023-11-20 15:40:10.111780, updated_at=2023-11-20 15:40:10.111780)
   url=https://pre.argilla.io/dataset/c082ca28-49d1-4b25-a41d-6fc143dd1e63/annotation-mode
   fields=[RemoteTextField(id=UUID('c2574040-7bae-4c77-9b2b-1166b87a58be'), client=None, name='prompt', title='Prompt', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('8f0be964-bcf0-4095-88b1-eb9ee2720d6e'), client=None, name='response', title='Response', required=True, type='text', use_markdown=True)]
   questions=[RemoteRatingQuestion(id=UUID('03d37870-69fa-4594-a1c7-c89eb972d0f6'), client=None, name='prompt-quality', title='Prompt Quality', description=None, required=True, type='rating', values=[1, 2, 3, 4, 5, 6, 7]), RemoteLabelQuestion(id=UUID('18f1e91c-a7d3-4d24-a9a9-922701e1f3b1'), client=None, name='prompt-inten

#### Configure the metadata-properties

Next we will be using `text-descriptives` to configure the metadata-properties. This will be used to add and updat relevant metadata-properties to the dataset. Because `text-descriptives` doesn't provide any programmatic interface with the metrics-groups and their sub-metrics, we will run the computation on the an example text and use the results to configure the metadata-properties.

In [8]:
metric_group = ["descriptive_stats"]
relevant_subgroups = []
df_metrics = td.extract_metrics(
    text=["this is an example prompt"], 
    lang="en", 
    metrics=metric_group,
    spacy_model="en_core_web_sm"
).drop(columns=["text"] + relevant_subgroups if relevant_subgroups else ["text"])
df_metrics.columns

ℹ Both a spacy model and a language were provided. Will use the spacy
model and ignore language.


Index(['token_length_mean', 'token_length_median', 'token_length_std',
       'sentence_length_mean', 'sentence_length_median', 'sentence_length_std',
       'syllables_per_token_mean', 'syllables_per_token_median',
       'syllables_per_token_std', 'n_tokens', 'n_unique_tokens',
       'proportion_unique_tokens', 'n_characters', 'n_sentences'],
      dtype='object')

Next, we will be working on converting the `text-descriptives` output to a format that can be used to configure the metadata-properties for our supported types: `TermsMetadataProperty`, `IntegerMetadataProperty` and `FloatMetadataProperty`. Note that we are also applying some subjective formatting choices to ensure that the metadata-properties are easy to read and understand.

In [11]:
def clean_column_name(col_name):
    """Clean a column name to fit a specific regex pattern."""
    col_name = col_name.lower()  # Convert to lowercase
    col_name = re.sub(r'[^a-z0-9_]', '_', col_name)  # Replace non-alphanumeric characters with underscores
    return col_name

def create_metadata_properties(df, prefix):
    """Generate metadata properties based on dataframe columns and data types."""
    properties = []
    for col, dtype in df.dtypes.items():
        name = f"{prefix}_{clean_column_name(col)}"
        title = name.replace('_', ' ').title()

        if dtype == 'object':
            prop = rg.TermsMetadataProperty(name=name, title=title)
        elif dtype == 'int64':
            prop = rg.IntegerMetadataProperty(name=name, title=title)
        elif dtype == 'float64':
            prop = rg.FloatMetadataProperty(name=name, title=title)
        elif dtype == 'bool':
            prop = rg.TermsMetadataProperty(name=name, title=title)
        else:
            print(f"Unhandled data type for column {col}: {dtype}")
            continue
        properties.append(prop)
    return properties

metadata_properties = []
metadata_properties += create_metadata_properties(df_metrics, 'prompt')
metadata_properties += create_metadata_properties(df_metrics, 'response')
for metadata_property in metadata_properties:
    try:
        field = ds_remote.metadata_property_by_name(metadata_property.name)
        if not field:
            ds_remote.add_metadata_property(metadata_property)
    except (KeyError, ValueError) as e:
        ds_remote.add_metadata_property(metadata_property)        
ds_remote.metadata_properties

[RemoteFloatMetadataProperty(id=UUID('e46f3316-1ec4-4777-95aa-e379859ab520'), client=<httpx.Client object at 0x1379bd940>, name='prompt_token_length_mean', title='Prompt Token Length Mean', visible_for_annotators=True, type='float', min=None, max=None),
 RemoteFloatMetadataProperty(id=UUID('3f186741-7948-47ff-a048-54f4b523b076'), client=<httpx.Client object at 0x1379bd940>, name='prompt_token_length_median', title='Prompt Token Length Median', visible_for_annotators=True, type='float', min=None, max=None),
 RemoteFloatMetadataProperty(id=UUID('eea19ccc-39f1-4f2a-87fa-5f011a4004e8'), client=<httpx.Client object at 0x1379bd940>, name='prompt_token_length_std', title='Prompt Token Length Std', visible_for_annotators=True, type='float', min=None, max=None),
 RemoteFloatMetadataProperty(id=UUID('e07ddb06-e234-40f3-b802-5f7bacea156e'), client=<httpx.Client object at 0x1379bd940>, name='prompt_sentence_length_mean', title='Prompt Sentence Length Mean', visible_for_annotators=True, type='float

### Data collection

In [12]:
dataset = load_dataset("zetavg/ShareGPT-Processed")
dataset = dataset["train"]
dataset

Dataset({
    features: ['id', 'conversations', 'lang'],
    num_rows: 90665
})

In [13]:
dataset = dataset.filter(function=lambda x: x.get("lang", "?") == "en")
dataset = dataset.filter(lambda x: x["conversations"][0]["from"] == "human")
dataset = dataset.filter(lambda x: len(x["conversations"])>1)
dataset

Filter:   0%|          | 0/63940 [00:00<?, ? examples/s]

Filter: 100%|██████████| 63643/63643 [00:10<00:00, 6348.19 examples/s]


Dataset({
    features: ['id', 'conversations', 'lang'],
    num_rows: 62338
})

In [16]:
dataset = dataset.shuffle(seed=42)
dataset = dataset.select(list(range(min(10000, len(dataset)))))
dataset

Dataset({
    features: ['id', 'conversations', 'lang'],
    num_rows: 10000
})

In [17]:
dataset = dataset.map(lambda x: {"prompt": x["conversations"][0]["value"], "response": x["conversations"][1]["value"]})
dataset

Map: 100%|██████████| 10000/10000 [00:02<00:00, 3831.43 examples/s]


Dataset({
    features: ['id', 'conversations', 'lang', 'prompt', 'response'],
    num_rows: 10000
})

In [18]:
# Extract metrics
spacy_model = "en_core_web_md" # we need a model with vectors
df_prompt = td.extract_metrics(text=dataset["prompt"], metrics=metric_group, spacy_model=spacy_model).drop(columns=['text'])
df_response = td.extract_metrics(text=dataset["response"], metrics=metric_group, spacy_model=spacy_model).drop(columns=['text'])

# Identify integer and boolean columns for prompts and responses
int_cols_prompts = df_prompt.select_dtypes(include=['int64']).columns.tolist()
bool_cols_prompts = df_prompt.select_dtypes(include=['boolean']).columns.tolist()

int_cols_responses = df_response.select_dtypes(include=['int64']).columns.tolist()
bool_cols_responses = df_response.select_dtypes(include=['boolean']).columns.tolist()

# Combine column lists for prompts and responses
int_cols = list(set(int_cols_prompts + int_cols_responses))
bool_cols = list(set(bool_cols_prompts + bool_cols_responses))
int_cols, bool_cols

(['n_sentences', 'n_unique_tokens', 'n_characters', 'n_tokens'], [])

Next, we will be casting the `numpy`-datatypes to basic Python built-in datatypes. This is required because the Argilla client doesn't support `numpy`-datatypes.

In [19]:
# --- Functions ---
def cast_to_python_types(df):
    """
    Convert integer and boolean columns to Python native types.
    """
    for column in df.columns:
        df[column].fillna(0, inplace=True)
        if df[column].dtype == bool:
            df[column] = df[column].astype(str)
        elif df[column].dtype == np.int64:
            df[column] = df[column].astype(int)
        elif df[column].dtype == np.float64:
            df[column] = df[column].astype(float)
        else:
            print(f"Unhandled data type for column {column}: {df[column].dtype}")
    return df

df_prompt = cast_to_python_types(df_prompt)
df_response = cast_to_python_types(df_response)

Lastly, we will loop through the Hugging Face dataset, add the metadata-properties and update the Argilla dataset with the new records.

In [21]:
# Prepare feedback records with metadata and suggestions
records = []

cols_with_values_other_than_zeros_or_nan_prompt = df_prompt.columns[~(df_prompt.fillna(0) == 0).all() & ~df_prompt.isnull().any()].tolist()
cols_with_values_other_than_zeros_or_nan_response = df_response.columns[~(df_response.fillna(0) == 0).all() & ~df_response.isnull().any()].tolist()


for i, record in enumerate(dataset):
    # Prepare metadata for prompts
    metadata_prompts = {f"prompt_{col}": value for col, value in df_prompt[cols_with_values_other_than_zeros_or_nan_prompt].iloc[i].items()}
    # Prepare metadata for responses
    metadata_response = {f"response_{col}": value for col, value in df_response[cols_with_values_other_than_zeros_or_nan_response].iloc[i].items()}

    # Explicitly cast integers using Python's native int type
    for col in int_cols:
        if f"prompt_{col}" in metadata_prompts:
            metadata_prompts[f"prompt_{col}"] = int(metadata_prompts[f"prompt_{col}"])
        if f"response_{col}" in metadata_response:
            metadata_response[f"response_{col}"] = int(metadata_response[f"response_{col}"])

    # Convert booleans to strings using Python's native str type
    for col in bool_cols:
        if f"prompt_{col}" in metadata_prompts:
            metadata_prompts[f"prompt_{col}"] = str(metadata_prompts[f"prompt_{col}"])
        if f"response_{col}" in metadata_response:
            metadata_response[f"response_{col}"] = str(metadata_response[f"response_{col}"])

    # Combine both metadata dictionaries into one
    metadata = {**metadata_prompts, **metadata_response}
    record = rg.FeedbackRecord(
        fields={"prompt": record["prompt"], "response": record["response"]},
        metadata=metadata,
    )
    records.append(record)

# Add records to the dataset and push to Argilla
ds_remote.add_records(records)

Pushing records to Argilla...: 100%|██████████| 313/313 [02:35<00:00,  2.02it/s]


Lastly, we will be adding some vectors to represent the `prompt` and `response` fields.

In [22]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://dmrepository.datamaran.com:8443/repository/dmPYTHON/simple
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 8.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.5 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.3 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 9.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 MB 7.9 MB/s eta 0:00:0000:0100:01
  Using cached https://dmrepository.datamaran.com:8443/repository/dmPYTHON/packages/networkx/3.2.1/networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached https://dmrepository.datamaran.com:8443/repository/dmPYTHON/packages/sympy/1.12/sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached https://dmrepository.datamaran.com:8443/repository/dmPYTHON/packages/threadpoolctl/3.2.0/threa

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('TaylorAI/bge-micro-v2')

In [8]:
try:
    prompt_setting = ds_remote.vector_settings_by_name("prompt")
    response_setting = ds_remote.vector_settings_by_name("response")
except Exception as e:
    prompt_setting = ds_remote.add_vector_settings(rg.VectorSettings(name="prompt", dimensions=384))
    response_setting = ds_remote.add_vector_settings(rg.VectorSettings(name="response", dimensions=384))
prompt_setting, response_setting

(RemoteVectorSettings(name='prompt', title='Prompt', dimensions=384, id=UUID('83bdef3b-f787-4601-8df8-2bc7eeb8b6a2'), inserted_at=datetime.datetime(2023, 11, 20, 17, 53, 16, 520211), updated_at=datetime.datetime(2023, 11, 20, 17, 53, 16, 520211)),
 RemoteVectorSettings(name='response', title='Response', dimensions=384, id=UUID('34705247-38cf-45ba-bd52-fb8642297814'), inserted_at=datetime.datetime(2023, 11, 20, 17, 53, 17, 35059), updated_at=datetime.datetime(2023, 11, 20, 17, 53, 17, 35059)))

In [11]:
modified_records = []
prompt_text = []
response_text = []
for record in ds_remote.records:
    prompt_text.append(record.fields["prompt"])
    response_text.append(record.fields["response"])
    modified_records.append(record)

In [12]:
prompt_vectors = model.encode(prompt_text)
response_vectors = model.encode(response_text)

In [17]:
for record, prompt_vector, response_vector in zip(modified_records, prompt_vectors, response_vectors):
    record.vectors = {
        "prompt": prompt_vector.tolist(),
        "response": response_vector.tolist(),
    }

In [27]:
import numpy as np
import time
chunked_modified_records = np.array_split(modified_records, 20)
while len(chunked_modified_records) > 0:
    try:
        ds_remote.update_records(chunked_modified_records[0])
        chunked_modified_records.pop(0)
    except Exception as e:
        print(len(chunked_modified_records))
        time.sleep(60)


20


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│    3 chunked_modified_records = np.array_split(modified_records, 20)                             │
│    4 while len(chunked_modified_records) > 0:                                                    │
│    5 │   try:                                                                                    │
│ ❱  6 │   │   ds_remote.update_records(chunked_modified_records[0])                               │
│    7 │   │   chunked_modified_records.pop(0)                                                     │
│    8 │   except Exception as e:                                                                  │
│    9 │   │   print(len(chunked_modified_records))                                                │
│                                                                                                  │
│ /Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9 │
│ /site-packages/argilla/client/feedback/dataset/remote/dataset.py:630 in update_records           │
│                                                                                                  │
│    627 │   │   Raises:                                                                           │
│    628 │   │   │   PermissionError: if the user does not have either `owner` or `admin` role.    │
│    629 │   │   """                                                                               │
│ ❱  630 │   │   self._records.update(records=records, show_progress=show_progress)                │
│    631 │                                                                                         │
│    632 │   def delete_records(self, records: Union["RemoteFeedbackRecord", List["RemoteFeedback  │
│    633 │   │   """Deletes the given records from the dataset in Argilla.                         │
│                                                                                                  │
│ /Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9 │
│ /site-packages/argilla/client/utils.py:55 in wrapper                                             │
│                                                                                                  │
│    52 │   │   @functools.wraps(func)                                                             │
│    53 │   │   def wrapper(*args: _P.args, **kwargs: _P.kwargs) -> _R:                            │
│    54 │   │   │   client = args[0]._client if hasattr(args[0], "_client") else active_client()   │
│ ❱  55 │   │   │   user = users_api.whoami_httpx(client).parsed                                   │
│    56 │   │   │   if user.role not in roles:                                                     │
│    57 │   │   │   │   raise PermissionError(                                                     │
│    58 │   │   │   │   │   f"User with role={user.role} is not allowed to call `{func.__name__}   │
│                                                                                                  │
│ /Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9 │
│ /site-packages/argilla/client/sdk/users/api.py:67 in whoami_httpx                                │
│                                                                                                  │
│    64 │   │   │   headers=response.headers,                                                      │
│    65 │   │   │   parsed=parsed_response,                                                        │
│    66 │   │   )                                                                                  │
│ ❱  67 │   return handle_response_error(response)           